#Split data into Training and Test sets

The first step in order to build our model for the Iris dataset, given that initial EDA did not indicate the need for further data preprocessing, is to split our dataset. We'll fit our model on the training set, and test it on the test set.  For this purpose we'll use scikit-learn's built-in "train_test_split" function. This function shuffles and splits our initial dataset into a training set with 75% of the samples, and a tess set with the remaining 30% of our observations.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
iris_dataset = load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris_dataset['data'], iris_dataset['target'], random_state = 1)